# 99 — Lab Notes & Learning Journal
## Reflections on the Character-Level LSTM Project

---


## 📖 Purpose

This notebook is your **learning journal** for the Frankenstein text generation project.

Use it to:
- Document what you learned
- Track experiments and results
- Note confusions and breakthroughs
- Plan next steps

**Reflections transform activity into understanding.** Take time to write thoughtfully.

---


## 📅 Session Log

### Session 1: [Date]

**Goal:**  
*What did you set out to accomplish?*

**What I Did:**  
*List notebooks completed, experiments run*

**What Clicked:**  
*Concepts that finally made sense*

**What Confused Me:**  
*Questions or concepts still unclear*

**Training Loss:**  
*Epoch 1: X.XXX | Epoch 2: X.XXX | ... | Epoch 5: X.XXX*

**Generated Text Quality:**  
*Brief assessment: coherent? repetitive? style match?*

**Time Spent:**  
*Rough estimate*

---


### Session 2: [Date]

**Experiment:**  
*e.g., Tried temperature sampling with T=0.8*

**Result:**  
*What happened?*

**Observation:**  
*What did you notice?*

**Next Action:**  
*What will you try next?*

---


## 🧠 Key Concepts Learned

### Character-Level Tokenization
*In your own words, what is it and why use it?*

### LSTM Gates
*What do the three gates (forget, input, output) actually control?*

### Hidden State vs. Cell State
*What's the difference and why do we need both?*

### Training vs. Generation
*How do these two phases differ?*

### Why Reshape for CrossEntropyLoss
*Why do we flatten from [B, T, V] to [B*T, V]?*

---


## 📊 Results Summary

### Training Metrics

| Epoch | Loss | Notes |
|-------|------|-------|
| 1 | | |
| 2 | | |
| 3 | | |
| 4 | | |
| 5 | | |

### Generated Text Sample

**Prompt:** "You will rejoice to hear"

**Output (first 200 chars):**
```
[Paste your generated text here]
```

**Assessment:**
- **Style match**: [1-5 scale] - How Frankenstein-like?
- **Coherence**: [1-5 scale] - Does it make sense?
- **Repetition**: [Yes/No] - Any loops or repeated phrases?

---


## 🔬 Experiments to Try

### Experiment Ideas

- [ ] **Temperature Sampling**: Try T = 0.5, 0.8, 1.2 — compare outputs
- [ ] **Longer Sequences**: Change `seq_length` from 48 to 64 or 96
- [ ] **Larger Hidden Size**: Increase from 96 to 128 or 256
- [ ] **More Epochs**: Train for 10 or 20 epochs instead of 5
- [ ] **Full Novel**: Train on entire Frankenstein text
- [ ] **Different Prompts**: Try various starting prompts
- [ ] **Beam Search**: Implement top-k sampling
- [ ] **Learning Rate Tuning**: Try lr = 0.001, 0.01, 0.05

### Experiment Log

#### Experiment: [Name]
**Hypothesis:**  
*What do you expect to happen?*

**Setup:**  
*What parameters changed?*

**Result:**  
*What actually happened?*

**Conclusion:**  
*What did you learn?*

---


## 💡 Insights & Breakthroughs

### What Surprised Me
*Unexpected results or behaviors*

### Biggest Challenge
*What was hardest to understand or implement?*

### Most Valuable Learning
*What concept or skill will stick with you?*

### Connection to Other Projects
*How does this relate to other ML/DL concepts you've learned?*

---


## 🚀 Next Steps

### Short-Term (This Week)
1. 
2. 
3. 

### Medium-Term (This Month)
1. 
2. 
3. 

### Long-Term (This Quarter)
1. 
2. 
3. 

---


## 📚 Resources & References

### Helpful Links
- [PyTorch LSTM Documentation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [The Unreasonable Effectiveness of RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Similar Projects to Explore
- Word-level language models
- Text classification with LSTM
- Sequence-to-sequence models
- Transformer-based text generation (GPT-style)

---


## ✨ Final Reflection

### What This Project Taught Me

### How I Grew as a Machine Learning Practitioner

### What I Would Do Differently Next Time

### My Favorite Part of This Project

---

*Keep learning, keep building, keep reflecting.* 🚀
